<a href="https://colab.research.google.com/github/ShreyG12345/DATA70202/blob/main/Linear_%26_Random_Forest_Male_%26_Female_Split_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this code is sensitive and must not be shared publicly and can only be viewed by instructors.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Load Datasets
female_data = pd.read_csv("female.csv")
male_data = pd.read_csv("male.csv")

# Define predictors and response variable
predictors = [
    'active_service_on_full_time_basis_2015_scheme',
    'pensionable_service_final_salary_scheme',
    'age_at_join',
    'years_until_state_pension',
    'rank',
    'part_time_proportion'
]
response = 'actual_pay_over_12_months_final_salary_scheme'


# ----------------------------------------------------------
# LINEAR REGRESSION

# Function to evaluate model and return predictions, residuals, metrics, and model
def evaluate_model(df, predictors, response):
    X = df[predictors].dropna()
    y = df.loc[X.index, response]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return y_test, y_pred, residuals, r2, rmse


# Function to compute scaled coefficients
def get_scaled_coefficients(df, predictors, response):
    X = df[predictors].dropna()
    y = df.loc[X.index, response]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    model = LinearRegression()
    model.fit(X_scaled, y)
    coef_df = pd.DataFrame({
        "Feature": predictors,
        "Coefficient": model.coef_,
        "Abs_Coefficient": abs(model.coef_)
    }).sort_values(by="Abs_Coefficient", ascending=False)
    return coef_df

# Evaluate models
female_y_test, female_y_pred, female_residuals, female_r2, female_rmse = evaluate_model(female_data, predictors, response)
male_y_test, male_y_pred, male_residuals, male_r2, male_rmse = evaluate_model(male_data, predictors, response)

# Get coefficients
female_coef_df = get_scaled_coefficients(female_data, predictors, response)
male_coef_df = get_scaled_coefficients(male_data, predictors, response)

# Plot 1: Plot predicted vs actual for male and female side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.scatterplot(x=female_y_pred, y=female_y_test, alpha=0.3, ax=axs[0])
axs[0].plot([female_y_test.min(), female_y_test.max()], [female_y_test.min(), female_y_test.max()], '--r')
axs[0].set_title(f'Female: Predicted vs Actual\nR²={female_r2:.3f}, RMSE={female_rmse:.2f}')
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual")

sns.scatterplot(x=male_y_pred, y=male_y_test, alpha=0.3, ax=axs[1])
axs[1].plot([male_y_test.min(), male_y_test.max()], [male_y_test.min(), male_y_test.max()], '--r')
axs[1].set_title(f'Male: Predicted vs Actual\nR²={male_r2:.3f}, RMSE={male_rmse:.2f}')
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual")
plt.tight_layout()
plt.show()

# Plot 2: Plot residuals vs predicted for male and female side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.scatterplot(x=female_y_pred, y=female_residuals, color='blue', alpha=0.3, ax=axs[0])
axs[0].axhline(0, color='red', linestyle='--')
axs[0].set_title("Female: Residuals vs Predicted")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Residuals")

sns.scatterplot(x=male_y_pred, y=male_residuals, color='blue', alpha=0.3, ax=axs[1])
axs[1].axhline(0, color='red', linestyle='--')
axs[1].set_title("Male: Residuals vs Predicted")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Residuals")
plt.tight_layout()
plt.show()

# Plot 3: Plot histogram of residuals for male and female side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.histplot(female_residuals, kde=True, bins=40, color='red', ax=axs[0])
axs[0].set_title("Female: Residuals Distribution")
axs[0].set_xlabel("Residuals")
axs[0].set_ylabel("Frequency")

sns.histplot(male_residuals, kde=True, bins=40, color='red', ax=axs[1])
axs[1].set_title("Male: Residuals Distribution")
axs[1].set_xlabel("Residuals")
axs[1].set_ylabel("Frequency")
plt.tight_layout()
plt.show()



# Plot 4: Plot of absolute values of coefficients for females
plt.figure(figsize=(10, 6))
sns.barplot(x="Abs_Coefficient", y="Feature", data=female_coef_df, palette="Reds_d")
plt.title("Linear Regression Coefficients (Female)", fontsize=16)
plt.xlabel("Absolute Coefficient Value", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()

# Plot 5: Plot of absolute values of coefficients for males
plt.figure(figsize=(10, 6))
sns.barplot(x="Abs_Coefficient", y="Feature", data=male_coef_df, palette="Blues_d")
plt.title("Linear Regression Coefficients (Male)", fontsize=16)
plt.xlabel("Absolute Coefficient Value", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()


# Plot 6: Q-Q plots for male and female side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
stats.probplot(female_residuals, dist="norm", plot=axs[0])
axs[0].set_title("Q-Q Plot: Female Residuals")

stats.probplot(male_residuals, dist="norm", plot=axs[1])
axs[1].set_title("Q-Q Plot: Male Residuals")

plt.tight_layout()
plt.show()


# Replot coefficients using absolute values for side-by-side comparison

# Combine using absolute coefficients
female_abs = female_coef_df[["Feature", "Abs_Coefficient"]].copy()
female_abs["Gender"] = "Female"

male_abs = male_coef_df[["Feature", "Abs_Coefficient"]].copy()
male_abs["Gender"] = "Male"

combined_abs_coef_df = pd.concat([female_abs, male_abs], ignore_index=True)

# Plot 7: Combined plot of the absolute coefficients of both men and women
plt.figure(figsize=(12, 6))
sns.barplot(data=combined_abs_coef_df, x="Abs_Coefficient", y="Feature", hue="Gender", orient="h", palette={"Male": 'dodgerblue', "Female": 'crimson'})
plt.title("Linear Regression Coefficients by Gender", fontsize=16)
plt.xlabel("Absolute Coefficient Value", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()




# --------------------------------------------------------------
# RANDOM FOREST

# Function to train Random Forest model and get feature importances
def train_rf_and_get_importance(df, predictors, response):
    X = df[predictors].dropna()
    y = df.loc[X.index, response]
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    importance_df = pd.DataFrame({
        "Feature": predictors,
        "Importance": model.feature_importances_
    })
    return model, importance_df

# Train models and collect importances
female_rf_model, female_rf_importance_df = train_rf_and_get_importance(female_data, predictors, response)
female_rf_importance_df["Gender"] = "Female"

male_rf_model, male_rf_importance_df = train_rf_and_get_importance(male_data, predictors, response)
male_rf_importance_df["Gender"] = "Male"

# Plot 1: Combine and plot the feature importances of male and females together
combined_rf_importance_df = pd.concat([female_rf_importance_df, male_rf_importance_df], ignore_index=True)

plt.figure(figsize=(12, 6))
sns.barplot(data=combined_rf_importance_df, x="Importance", y="Feature", hue="Gender", orient="h", palette={"Male": 'dodgerblue', "Female": 'crimson'})
plt.title("Random Forest Feature Importances by Gender", fontsize=16)
plt.xlabel("Importance Score", fontsize=14)
plt.ylabel("Feature", fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()



# Function to train RF model and evaluate with plots
def evaluate_rf_model(df, predictors, response):
    X = df[predictors].dropna()
    y = df.loc[X.index, response]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    return y_test, y_pred, residuals, r2, rmse

# Evaluate models
female_y_test_rf, female_y_pred_rf, female_residuals_rf, female_r2_rf, female_rmse_rf = evaluate_rf_model(female_data, predictors, response)
male_y_test_rf, male_y_pred_rf, male_residuals_rf, male_r2_rf, male_rmse_rf = evaluate_rf_model(male_data, predictors, response)

# Plot 2: Plot Predicted vs Actual for male and female side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.scatterplot(x=female_y_pred_rf, y=female_y_test_rf, alpha=0.3, ax=axs[0])
axs[0].plot([female_y_test_rf.min(), female_y_test_rf.max()], [female_y_test_rf.min(), female_y_test_rf.max()], '--r')
axs[0].set_title(f'Random Forest (Female): Predicted vs Actual\nR²={female_r2_rf:.3f}, RMSE={female_rmse_rf:.2f}')
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual")

sns.scatterplot(x=male_y_pred_rf, y=male_y_test_rf, alpha=0.3, ax=axs[1])
axs[1].plot([male_y_test_rf.min(), male_y_test_rf.max()], [male_y_test_rf.min(), male_y_test_rf.max()], '--r')
axs[1].set_title(f'Random Forest (Male): Predicted vs Actual\nR²={male_r2_rf:.3f}, RMSE={male_rmse_rf:.2f}')
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual")
plt.tight_layout()
plt.show()

# Plot 3: Plot Residuals vs Predicted for males and females side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.scatterplot(x=female_y_pred_rf, y=female_residuals_rf, color='blue', alpha=0.3, ax=axs[0])
axs[0].axhline(0, color='red', linestyle='--')
axs[0].set_title("Random Forest (Female): Residuals vs Predicted")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Residuals")

sns.scatterplot(x=male_y_pred_rf, y=male_residuals_rf, color='blue', alpha=0.3, ax=axs[1])
axs[1].axhline(0, color='red', linestyle='--')
axs[1].set_title("Random Forest (Male): Residuals vs Predicted")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Residuals")
plt.tight_layout()
plt.show()

# Plot 4: Plot Histogram of residuals for males and females side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
sns.histplot(female_residuals_rf, kde=True, bins=40, color='red', ax=axs[0])
axs[0].set_title("Random Forest (Female): Residuals Distribution")
axs[0].set_xlabel("Residuals")
axs[0].set_ylabel("Frequency")

sns.histplot(male_residuals_rf, kde=True, bins=40, color='red', ax=axs[1])
axs[1].set_title("Random Forest (Male): Residuals Distribution")
axs[1].set_xlabel("Residuals")
axs[1].set_ylabel("Frequency")
plt.tight_layout()
plt.show()




# Define stable features for the female partial dependence plots (PDPs). These are the features
# that do not have low variation for females.
stable_features_f = [
    'active_service_on_full_time_basis_2015_scheme',
    'pensionable_service_final_salary_scheme',
    'age_at_join',
    'years_until_state_pension',
    'rank',
    'part_time_proportion'
]

# Define stable features for the male partial dependence plots (PDPs)
stable_features_m = [
    'active_service_on_full_time_basis_2015_scheme',
    'pensionable_service_final_salary_scheme',
    'age_at_join',
    'years_until_state_pension',
    'rank'
]

# Extract features from female and male data
X_female = female_data[stable_features_f].dropna()
X_male = male_data[stable_features_m].dropna()

# PDP for female model
fig_female, ax_female = plt.subplots(figsize=(10, 8))
PartialDependenceDisplay.from_estimator(
    female_rf_model,
    X_female,
    stable_features_f,
    ax=ax_female,
    kind="average",
    grid_resolution=50
)
ax_female.set_title("Partial Dependence Plots - Female Model")
plt.tight_layout()
plt.show()

# PDP for male model
fig_male, ax_male = plt.subplots(figsize=(10, 8))
PartialDependenceDisplay.from_estimator(
    male_rf_model,
    X_male,
    stable_features_m,
    ax=ax_male,
    kind="average",
    grid_resolution=50
)
ax_male.set_title("Partial Dependence Plots - Male Model")
plt.tight_layout()
plt.show()
plt.show()







